# Classification Exercise

We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belogn in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

## Follow the Directions in Bold. If you get stuck, check out the solutions lecture.

### THE DATA

** Read in the census_data.csv data with pandas**

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\PeterLi\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
dataset = pd.read_csv('census_data.csv')
dataset.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


** TensorFlow won't be able to understand strings as labels, you'll need to use pandas .apply() method to apply a custom function that converts them to 0s and 1s. This might be hard if you aren't very familiar with pandas, so feel free to take a peek at the solutions for this part.**

** Convert the Label column to 0s and 1s instead of strings.**

In [4]:
dataset['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [7]:
def lab_fix(label):
    if label == ' <=50K':
        return 0
    else:
        return 1

In [8]:
dataset['income_bracket'] = dataset['income_bracket'].apply(lab_fix)

### Perform a Train Test Split on the Data

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
x_data = dataset.drop('income_bracket', axis = 1)
y_labels = dataset['income_bracket']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_labels, test_size = 0.3,random_state = 101)

### Create the Feature Columns for tf.esitmator

** Take note of categorical vs continuous values! **

In [17]:
dataset.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

** Create the tf.feature_columns for the categorical values. Use vocabulary lists or just use hash buckets. **

In [42]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender",['Female','Male'])
education = tf.feature_column.categorical_column_with_hash_bucket("education",hash_bucket_size=100)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass",hash_bucket_size=1000)
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation",hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status",hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country",hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship",hash_bucket_size=1000)
race = tf.feature_column.categorical_column_with_hash_bucket("race",hash_bucket_size=100)

** Create the continuous feature_columns for the continuous values using numeric_column **

In [43]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")


** Put all these variables into a single list with the variable name feat_cols **

In [46]:
feat_cols = [age, workclass, education, education_num, marital_status,
       occupation, relationship, gender, capital_gain,
       capital_loss, hours_per_week, native_country, race]

### Create Input Function

** Batch_size is up to you. But do make sure to shuffle!**

In [36]:
input_func = tf.estimator.inputs.pandas_input_fn(x = X_train, y = y_train,
                                                 batch_size = 100,
                                                num_epochs = 1000,
                                                shuffle = True)

#### Create your model with tf.estimator

**Create a LinearClassifier.(If you want to use a DNNClassifier, keep in mind you'll need to create embedded columns out of the cateogrical feature that use strings, check out the previous lecture on this for more info.)**

In [48]:
model = tf.estimator.LinearClassifier(feature_columns = feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\PeterLi\\AppData\\Local\\Temp\\tmp52lqlqos', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000018168633C88>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


** Train your model on the data, for at least 5000 steps. **

In [49]:
model.train(input_fn = input_func)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\PeterLi\AppData\Local\Temp\tmp52lqlqos\model.ckpt.
INFO:tensorflow:loss = 69.31472, step = 1
INFO:tensorflow:global_step/sec: 56.9378
INFO:tensorflow:loss = 692.71924, step = 101 (1.755 sec)
INFO:tensorflow:global_step/sec: 68.4888
INFO:tensorflow:loss = 90.69135, step = 201 (1.460 sec)
INFO:tensorflow:global_step/sec: 68.1627
INFO:tensorflow:loss = 92.279495, step = 301 (1.467 sec)
INFO:tensorflow:global_step/sec: 66.2269
INFO:tensorflow:loss = 65.69995, step = 401 (1.509 sec)
INFO:tensorflow:global_step/sec: 68.024
INFO:tensorflow:loss = 163.86972, step = 501 (1.471 sec)
INFO:tensorflow:global_step/sec: 66.4022
INFO:tensorflow:loss = 502.64218, step = 601 (1.508 sec)
INFO:tensorflow:global_step/sec: 66.9789
INFO:tensorflow:loss = 100.65004, step = 701 (1.491 sec)
INFO:tensorflow:global_step/sec: 68.3487
INFO:tensorflow:loss = 30.172012, step = 801 (1.464 sec)
INFO:tensorflow:global_step

INFO:tensorflow:loss = 78.55693, step = 8201 (1.506 sec)
INFO:tensorflow:global_step/sec: 68.6294
INFO:tensorflow:loss = 66.42498, step = 8301 (1.458 sec)
INFO:tensorflow:global_step/sec: 65.5771
INFO:tensorflow:loss = 62.398033, step = 8401 (1.524 sec)
INFO:tensorflow:global_step/sec: 68.7235
INFO:tensorflow:loss = 40.461193, step = 8501 (1.455 sec)
INFO:tensorflow:global_step/sec: 67.0685
INFO:tensorflow:loss = 50.542057, step = 8601 (1.491 sec)
INFO:tensorflow:global_step/sec: 66.2706
INFO:tensorflow:loss = 82.172386, step = 8701 (1.510 sec)
INFO:tensorflow:global_step/sec: 69.8242
INFO:tensorflow:loss = 33.79692, step = 8801 (1.433 sec)
INFO:tensorflow:global_step/sec: 68.3952
INFO:tensorflow:loss = 38.939228, step = 8901 (1.460 sec)
INFO:tensorflow:global_step/sec: 67.4294
INFO:tensorflow:loss = 17.742785, step = 9001 (1.485 sec)
INFO:tensorflow:global_step/sec: 67.4747
INFO:tensorflow:loss = 39.68139, step = 9101 (1.480 sec)
INFO:tensorflow:global_step/sec: 69.0548
INFO:tensorflo

INFO:tensorflow:global_step/sec: 66.845
INFO:tensorflow:loss = 69.77902, step = 16501 (1.495 sec)
INFO:tensorflow:global_step/sec: 65.5343
INFO:tensorflow:loss = 192.90475, step = 16601 (1.526 sec)
INFO:tensorflow:global_step/sec: 68.2091
INFO:tensorflow:loss = 24.381435, step = 16701 (1.466 sec)
INFO:tensorflow:global_step/sec: 67.5202
INFO:tensorflow:loss = 28.800756, step = 16801 (1.481 sec)
INFO:tensorflow:global_step/sec: 67.7026
INFO:tensorflow:loss = 29.91148, step = 16901 (1.478 sec)
INFO:tensorflow:global_step/sec: 67.5657
INFO:tensorflow:loss = 42.701992, step = 17001 (1.480 sec)
INFO:tensorflow:global_step/sec: 66.8005
INFO:tensorflow:loss = 40.521824, step = 17101 (1.498 sec)
INFO:tensorflow:global_step/sec: 68.7234
INFO:tensorflow:loss = 76.806435, step = 17201 (1.453 sec)
INFO:tensorflow:global_step/sec: 68.1164
INFO:tensorflow:loss = 154.1131, step = 17301 (1.468 sec)
INFO:tensorflow:global_step/sec: 66.0959
INFO:tensorflow:loss = 44.21309, step = 17401 (1.513 sec)
INFO:

INFO:tensorflow:loss = 34.399742, step = 24701 (1.579 sec)
INFO:tensorflow:global_step/sec: 62.1236
INFO:tensorflow:loss = 231.54456, step = 24801 (1.610 sec)
INFO:tensorflow:global_step/sec: 65.9858
INFO:tensorflow:loss = 23.340685, step = 24901 (1.514 sec)
INFO:tensorflow:global_step/sec: 55.3659
INFO:tensorflow:loss = 45.165844, step = 25001 (1.806 sec)
INFO:tensorflow:global_step/sec: 65.0242
INFO:tensorflow:loss = 44.23726, step = 25101 (1.539 sec)
INFO:tensorflow:global_step/sec: 66.0089
INFO:tensorflow:loss = 33.103245, step = 25201 (1.514 sec)
INFO:tensorflow:global_step/sec: 61.7791
INFO:tensorflow:loss = 33.683212, step = 25301 (1.620 sec)
INFO:tensorflow:global_step/sec: 68.2555
INFO:tensorflow:loss = 32.186943, step = 25401 (1.464 sec)
INFO:tensorflow:global_step/sec: 68.9598
INFO:tensorflow:loss = 27.19921, step = 25501 (1.450 sec)
INFO:tensorflow:global_step/sec: 67.9318
INFO:tensorflow:loss = 33.40858, step = 25601 (1.472 sec)
INFO:tensorflow:global_step/sec: 67.0687
INF

INFO:tensorflow:global_step/sec: 64.2144
INFO:tensorflow:loss = 45.145008, step = 33001 (1.557 sec)
INFO:tensorflow:global_step/sec: 64.2328
INFO:tensorflow:loss = 17.96882, step = 33101 (1.557 sec)
INFO:tensorflow:global_step/sec: 63.6009
INFO:tensorflow:loss = 25.681505, step = 33201 (1.574 sec)
INFO:tensorflow:global_step/sec: 62.4321
INFO:tensorflow:loss = 39.865486, step = 33301 (1.600 sec)
INFO:tensorflow:global_step/sec: 63.5237
INFO:tensorflow:loss = 106.942665, step = 33401 (1.573 sec)
INFO:tensorflow:global_step/sec: 63.9298
INFO:tensorflow:loss = 34.99444, step = 33501 (1.566 sec)
INFO:tensorflow:global_step/sec: 61.1375
INFO:tensorflow:loss = 37.664833, step = 33601 (1.636 sec)
INFO:tensorflow:global_step/sec: 61.9492
INFO:tensorflow:loss = 45.154198, step = 33701 (1.614 sec)
INFO:tensorflow:global_step/sec: 65.2358
INFO:tensorflow:loss = 34.19679, step = 33801 (1.532 sec)
INFO:tensorflow:global_step/sec: 65.4898
INFO:tensorflow:loss = 35.298935, step = 33901 (1.527 sec)
IN

...

INFO:tensorflow:global_step/sec: 67.101
INFO:tensorflow:loss = 43.705997, step = 188601 (1.490 sec)
INFO:tensorflow:global_step/sec: 65.969
INFO:tensorflow:loss = 28.686422, step = 188701 (1.516 sec)
INFO:tensorflow:global_step/sec: 66.9835
INFO:tensorflow:loss = 41.73881, step = 188801 (1.492 sec)
INFO:tensorflow:global_step/sec: 65.3266
INFO:tensorflow:loss = 34.944977, step = 188901 (1.532 sec)
INFO:tensorflow:global_step/sec: 66.0812
INFO:tensorflow:loss = 35.97807, step = 189001 (1.513 sec)
INFO:tensorflow:global_step/sec: 66.2519
INFO:tensorflow:loss = 33.56673, step = 189101 (1.512 sec)
INFO:tensorflow:global_step/sec: 67.1502
INFO:tensorflow:loss = 37.31703, step = 189201 (1.487 sec)
INFO:tensorflow:global_step/sec: 66.7444
INFO:tensorflow:loss = 31.549568, step = 189301 (1.498 sec)
INFO:tensorflow:global_step/sec: 65.6591
INFO:tensorflow:loss = 23.948679, step = 189401 (1.522 sec)
INFO:tensorflow:global_step/sec: 67.6365
INFO:tensorflow:loss = 31.216791, step = 189501 (1.479 s

INFO:tensorflow:global_step/sec: 67.0084
INFO:tensorflow:loss = 39.54404, step = 196701 (1.491 sec)
INFO:tensorflow:global_step/sec: 65.8228
INFO:tensorflow:loss = 26.633175, step = 196801 (1.518 sec)
INFO:tensorflow:global_step/sec: 64.6136
INFO:tensorflow:loss = 28.505728, step = 196901 (1.549 sec)
INFO:tensorflow:global_step/sec: 67.7898
INFO:tensorflow:loss = 24.54588, step = 197001 (1.475 sec)
INFO:tensorflow:global_step/sec: 66.9044
INFO:tensorflow:loss = 37.853676, step = 197101 (1.496 sec)
INFO:tensorflow:global_step/sec: 67.3575
INFO:tensorflow:loss = 41.44517, step = 197201 (1.485 sec)
INFO:tensorflow:global_step/sec: 65.7274
INFO:tensorflow:loss = 41.563366, step = 197301 (1.521 sec)
INFO:tensorflow:global_step/sec: 65.671
INFO:tensorflow:loss = 49.279186, step = 197401 (1.522 sec)
INFO:tensorflow:global_step/sec: 67.6704
INFO:tensorflow:loss = 62.993256, step = 197501 (1.478 sec)
INFO:tensorflow:global_step/sec: 64.6315
INFO:tensorflow:loss = 27.204176, step = 197601 (1.549

INFO:tensorflow:global_step/sec: 67.0255
INFO:tensorflow:loss = 31.75487, step = 204901 (1.492 sec)
INFO:tensorflow:global_step/sec: 66.8385
INFO:tensorflow:loss = 59.363842, step = 205001 (1.496 sec)
INFO:tensorflow:global_step/sec: 65.3868
INFO:tensorflow:loss = 24.368427, step = 205101 (1.530 sec)
INFO:tensorflow:global_step/sec: 66.8699
INFO:tensorflow:loss = 36.92722, step = 205201 (1.494 sec)
INFO:tensorflow:global_step/sec: 67.0022
INFO:tensorflow:loss = 28.958942, step = 205301 (1.492 sec)
INFO:tensorflow:global_step/sec: 64.7447
INFO:tensorflow:loss = 33.61073, step = 205401 (1.546 sec)
INFO:tensorflow:global_step/sec: 66.8924
INFO:tensorflow:loss = 23.9048, step = 205501 (1.494 sec)
INFO:tensorflow:global_step/sec: 66.5942
INFO:tensorflow:loss = 32.714943, step = 205601 (1.505 sec)
INFO:tensorflow:global_step/sec: 66.3177
INFO:tensorflow:loss = 30.197596, step = 205701 (1.505 sec)
INFO:tensorflow:global_step/sec: 66.1143
INFO:tensorflow:loss = 30.652609, step = 205801 (1.514 

INFO:tensorflow:global_step/sec: 64.6542
INFO:tensorflow:loss = 37.59052, step = 213101 (1.547 sec)
INFO:tensorflow:global_step/sec: 67.441
INFO:tensorflow:loss = 34.788532, step = 213201 (1.483 sec)
INFO:tensorflow:global_step/sec: 67.8899
INFO:tensorflow:loss = 28.360193, step = 213301 (1.473 sec)
INFO:tensorflow:global_step/sec: 65.6221
INFO:tensorflow:loss = 38.149155, step = 213401 (1.524 sec)
INFO:tensorflow:global_step/sec: 67.1722
INFO:tensorflow:loss = 32.280437, step = 213501 (1.489 sec)
INFO:tensorflow:global_step/sec: 67.8112
INFO:tensorflow:loss = 34.39364, step = 213601 (1.475 sec)
INFO:tensorflow:global_step/sec: 67.1097
INFO:tensorflow:loss = 25.706062, step = 213701 (1.490 sec)
INFO:tensorflow:global_step/sec: 65.5914
INFO:tensorflow:loss = 33.2996, step = 213801 (1.525 sec)
INFO:tensorflow:global_step/sec: 66.281
INFO:tensorflow:loss = 33.42025, step = 213901 (1.509 sec)
INFO:tensorflow:global_step/sec: 68.3486
INFO:tensorflow:loss = 41.32495, step = 214001 (1.464 sec

INFO:tensorflow:global_step/sec: 67.2545
INFO:tensorflow:loss = 33.98787, step = 221301 (1.485 sec)
INFO:tensorflow:global_step/sec: 65.3923
INFO:tensorflow:loss = 139.71812, step = 221401 (1.529 sec)
INFO:tensorflow:global_step/sec: 68.2893
INFO:tensorflow:loss = 33.14276, step = 221501 (1.464 sec)
INFO:tensorflow:global_step/sec: 67.7011
INFO:tensorflow:loss = 33.51291, step = 221601 (1.477 sec)
INFO:tensorflow:global_step/sec: 65.1587
INFO:tensorflow:loss = 28.799286, step = 221701 (1.535 sec)
INFO:tensorflow:global_step/sec: 64.6045
INFO:tensorflow:loss = 31.639187, step = 221801 (1.548 sec)
INFO:tensorflow:global_step/sec: 65.6144
INFO:tensorflow:loss = 27.230545, step = 221901 (1.524 sec)
INFO:tensorflow:global_step/sec: 66.0893
INFO:tensorflow:loss = 33.273643, step = 222001 (1.513 sec)
INFO:tensorflow:global_step/sec: 66.0683
INFO:tensorflow:loss = 28.876087, step = 222101 (1.515 sec)
INFO:tensorflow:global_step/sec: 68.1852
INFO:tensorflow:loss = 48.80105, step = 222201 (1.467

### Evaluation

** Create a prediction input function. Remember to only supprt X_test data and keep shuffle=False. **

In [51]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x= X_test,
                                             batch_size =len(X_test),
                                              shuffle=False)

** Use model.predict() and pass in your input function. This will produce a generator of predictions, which you can then transform into a list, with list() **

In [52]:
pred_gen = model.predict(input_fn = pred_fn)
predictions = list(pred_gen)

INFO:tensorflow:Restoring parameters from C:\Users\PeterLi\AppData\Local\Temp\tmp52lqlqos\model.ckpt-227920


In [60]:
final_preds = [pred['class_ids'][0] for pred in predictions]
final_preds

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,


** Import classification_report from sklearn.metrics and then see if you can figure out how to use it to easily get a full report of your model's performance on the test data. **

In [56]:
from sklearn.metrics import classification_report

In [61]:
print(classification_report(y_test,final_preds))

             precision    recall  f1-score   support

          0       0.89      0.92      0.91      7436
          1       0.72      0.63      0.68      2333

avg / total       0.85      0.85      0.85      9769

